A place to test random stuff

In [ ]:
from monkey.model.classification_model.efficientnet_b0 import (
    EfficientNet_B0,
)
import torch
import torch.nn as nn
from torchsummary import summary
from monkey.model.utils import (
    get_classification_metrics,
    get_activation_function,
)
from monkey.model.loss_functions import get_loss_function

In [3]:
model = EfficientNet_B0(num_classes=2)
model = model.eval()
# summary(model, input_size=(3,32,32))
print(model)

EfficientNet_B0(
  (model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=

In [20]:
test = torch.randn(size=(4, 3, 32, 32), dtype=torch.float32)
gt = torch.tensor([0, 0, 0, 1], dtype=torch.long)

loss_fn = nn.CrossEntropyLoss()

weighted_ce = get_loss_function("Weighted_CrossEntropy")
weighted_ce.set_weight(torch.tensor([0.4, 0.6]))

pred = []

act = get_activation_function("softmax")

with torch.no_grad():
    out = model(test)
    print(out.shape)
    out = act(out)

    pred_labels = torch.argmax(out, dim=1).cpu().tolist()

    pred.extend(pred_labels)

    loss = loss_fn(out, gt)
    weighted_loss = weighted_ce.compute_loss(out, gt)

gt_labels = gt.cpu().tolist()

print(f"probs {out}")
print(out.shape)
print(f"gt {gt}")
print(gt.shape)
print(f"loss {loss}")
print(f"weighted loss {weighted_loss}")

print(f"pred labels: {pred_labels}")
print(f"true labels: {gt_labels}")

metrics = get_classification_metrics(gt_labels, pred_labels)
print(metrics)

torch.Size([4, 2])
probs tensor([[0.4889, 0.5111],
        [0.4885, 0.5115],
        [0.4888, 0.5112],
        [0.4894, 0.5106]])
torch.Size([4, 2])
gt tensor([0, 0, 0, 1])
torch.Size([4])
loss 0.6990070343017578
weighted loss 0.6971888542175293
pred labels: [1, 1, 1, 1]
true labels: [0, 0, 0, 1]
{'Balanced_Accuracy': 0.5, 'Precision': 0.25, 'Recall': 1.0, 'F1': 0.4}
